In [1]:
USE {
    dependencies("org.xerial:sqlite-jdbc:3.47.1.0")
}

%use kandy
%use dataframe

In [2]:
import java.sql.DriverManager

val connection = DriverManager.getConnection("jdbc:sqlite:data.db")
val repositories = DataFrame.readSqlTable(connection, "Repositories")

println(repositories.columnNames())
println(repositories.columnTypes())



[id, size, project_id, is_disabled, is_fork, is_in_maintenance, location, is_java, is_kotlin, is_android, is_config, youngest_commit, oldest_commit]
[kotlin.String, kotlin.Long, kotlin.String?, kotlin.Boolean, kotlin.Boolean?, kotlin.Boolean, kotlin.String, kotlin.Boolean?, kotlin.Boolean?, kotlin.Boolean?, kotlin.Boolean?, kotlin.String?, kotlin.String?]


In [3]:
println("Total repositories : ${repositories.rowsCount()}" )

val cumulativeRepos = repositories
    .add("year") {
        row ->
        val date = row["youngest_commit"] as? String
        date?.take(4)?.let {
            if (it.toInt() < 2020) "2019" else it
        } ?: "2019"
    }
    .groupBy("year")
    .aggregate {
        count() into "yearly_count"
    }
    .sortBy("year")
    .insert("yearly_cumulative") {  "yearly_count"<Int>()}.after("yearly_count")
    .cumSum("yearly_cumulative")

cumulativeRepos.plot {
    bars {
        x("year")
        y("yearly_count")
        fillColor = Color.PEACH

    }
    line {
        x("year"){
            axis.name = "Calendar year"
        }
        y("yearly_cumulative"){
            axis.name = "Number of repositories added"
        }
        color = Color.ORANGE

    }
    layout {
        subtitle = "Total number of repositories per year"
    }

}


Total repositories : 39987


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="oSkBrk"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"yearly_cumulative":[15785.0,16772.0,18837.0,21995.0,28053.0,38505.0,39987.0],
"year":["2019","2020","2021","2022","2023","2024","2025"],
"yearly_count":[15785.0,987.0,2065.0,3158.0,6058.0,10452.0,1482.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true,
"name":"Calendar year"
},{
"aesthetic":"y",
"name":"Number of repositories added",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"year",
"y":"yearly_count"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"fill":"#ffe5b4",
"data":{
}
},{
"mapping":{
"x":"year",
"y":"yearly_cumulative"
},
"stat":"identity",
"color":"#fc8452",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
}]
};
 var plotContainer = document.getElementById("oSkBrk");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2019 
 
 
 
 
 
 
 
 
 2020 
 
 
 
 
 
 
 
 
 2021 
 
 
 
 
 
 
 
 
 2022 
 
 
 
 
 
 
 
 
 2023 
 
 
 
 
 
 
 
 
 2024 
 
 
 
 
 
 
 
 
 2025 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5,000 
 
 
 
 
 
 
 10,000 
 
 
 
 
 
 
 15,000 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 25,000 
 
 
 
 
 
 
 30,000 
 
 
 
 
 
 
 35,000 
 
 
 
 
 
 
 40,000 
 
 
 
 
 
 
 
 
 Number of repositories added 
 
 
 
 
 Calendar year

In [32]:
val active_repositories = repositories
    .filter { it["is_disabled"] != 1 && it["is_in_maintenance"] != 1 && it["is_fork"] != 1 }
    .filter { it["oldest_commit"] != null && it["youngest_commit"] != null }

val config_active_repositories = active_repositories
    .filter { it["is_config"] != 1 }

val jvm_active_repositories = active_repositories
    .filter { it["is_java"] == 1 || it["is_android"] == 1 || it["is_kotlin"] == 1  }

val other_active_repositories = active_repositories.rowsCount() - config_active_repositories.rowsCount() - jvm_active_repositories.rowsCount()

val repository_type by columnOf("Configuration Repository", "JVM Repository", "Other")
val count by columnOf(config_active_repositories.rowsCount(), jvm_active_repositories.rowsCount(), other_active_repositories)
val df = dataFrameOf(repository_type, count)

df.plot {
    pie {
        slice(count)
        fillColor(repository_type) {
            scale = categorical(
                "Configuration Repository" to Color.ORANGE,
                "JVM Repository" to Color.PURPLE,
                "Other" to Color.GREEN
            )
        }
        size = 33.0
        hole = 0.8
        alpha = 0.8
    }
    layout {
        style(Style.Void)
    }
}


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="OhUysa"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"count":[5498.0,5497.0,13962.0],
"repository_type":["Configuration Repository","JVM Repository","Other"]
},
"kind":"plot",
"scales":[{
"aesthetic":"fill",
"values":["#fc8452","#9a60b4","#3ba272"],
"limits":["Configuration Repository","JVM Repository","Other"]
}],
"layers":[{
"hole":0.8,
"mapping":{
"slice":"count",
"fill":"repository_type"
},
"stat":"identity",
"size":33.0,
"sampling":"none",
"alpha":0.8,
"position":"identity",
"geom":"pie",
"data":{
}
}],
"theme":{
"name":"classic",
"axis":{
"blank":true
},
"line":{
"blank":true
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
}
};
 var plotContainer = document.getElementById("OhUysa");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 repository_type 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Configuration Repository 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 JVM Repository 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Other

In [5]:
val kotlin_active_repos = jvm_active_repositories
    .filter { it["is_kotlin"] != 0}

println("Found ${jvm_active_repositories.rowsCount()} JVM repos, of which ${kotlin_active_repos.rowsCount()} repos with Kotlin inside")


val cumulativeJvmRepos = jvm_active_repositories
    .add("year") {
            row ->
        val date = row["oldest_commit"] as? String
        date?.take(4)?.let {
            if (it.toInt() < 2020) "2019" else it
        } ?: "2019"
    }
    .groupBy("year")
    .aggregate {
        count() into "yearly_count"
    }
    .sortBy("year")
    .insert("yearly_cumulative") {  "yearly_count"<Int>()}.after("yearly_count")
    .cumSum("yearly_cumulative")

val cumulativeKotlinRepos = kotlin_active_repos
    .add("year") {
            row ->
        val date = row["oldest_commit"] as? String
        date?.take(4)?.let {
            if (it.toInt() < 2020) "2019" else it
        } ?: "2019"
    }
    .groupBy("year")
    .aggregate {
        count() into "yearly_count"
    }
    .sortBy("year")
    .insert("yearly_cumulative") {  "yearly_count"<Int>()}.after("yearly_count")
    .cumSum("yearly_cumulative")

//println(cumulativeJvmRepos.filter { it["year"] == "2019" }.rowsCount())

println("-------------")
cumulativeJvmRepos.forEach {
    println("${it["year"]} ${it["yearly_count"]} ${it["yearly_cumulative"]}")
}
println("-------------")
cumulativeKotlinRepos.forEach {
    println("${it["year"]} ${it["yearly_count"]} ${it["yearly_cumulative"]}")
}
println("-------------")



Found 5497 JVM repos, of which 435 repos with Kotlin inside
-------------
2019 2195 2195
2020 512 2707
2021 637 3344
2022 699 4043
2023 884 4927
2024 570 5497
-------------
2019 144 144
2020 33 177
2021 53 230
2022 47 277
2023 95 372
2024 63 435
-------------


In [9]:

val years = listOf("2019", "2020", "2021", "2022", "2023", "2024")
val jvmPlotValues = years.map { year -> cumulativeJvmRepos.filter { it["year"] == year }[0]["yearly_count"] }
val kotlinPlotValues = years.map { year -> cumulativeKotlinRepos.filter { it["year"] == year }[0]["yearly_count"] }


val dataset = dataFrameOf(
    "year" to years,
    "JVM" to jvmPlotValues,
    "Kotlin" to kotlinPlotValues,
).gather("JVM", "Kotlin").into("type", "amount")

dataset.groupBy("type").plot {
    layout.title = "Kotlin adoption in JVM repositories"
     bars {
            x("year")
            y("amount")
            fillColor("type") {
                scale = categorical(
                    "JVM" to Color.ORANGE,
                    "Kotlin" to Color.PURPLE,
                )
            }
        }
}



<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="MEe4cT"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Kotlin adoption in JVM repositories"
},
"mapping":{
},
"data":{
"amount":[2195.0,512.0,637.0,699.0,884.0,570.0,144.0,33.0,53.0,47.0,95.0,63.0],
"&merged_groups":["JVM","JVM","JVM","JVM","JVM","JVM","Kotlin","Kotlin","Kotlin","Kotlin","Kotlin","Kotlin"],
"year":["2019","2020","2021","2022","2023","2024","2019","2020","2021","2022","2023","2024"],
"type":["JVM","JVM","JVM","JVM","JVM","JVM","Kotlin","Kotlin","Kotlin","Kotlin","Kotlin","Kotlin"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"values":["#fc8452","#9a60b4"],
"limits":["JVM","Kotlin"]
}],
"layers":[{
"mapping":{
"x":"year",
"y":"amount",
"fill":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("MEe4cT");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2019 
 
 
 
 
 
 
 
 
 2020 
 
 
 
 
 
 
 
 
 2021 
 
 
 
 
 
 
 
 
 2022 
 
 
 
 
 
 
 
 
 2023 
 
 
 
 
 
 
 
 
 2024 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 
 
 Kotlin adoption in JVM repositories 
 
 
 
 
 amount 
 
 
 
 
 year 
 
 
 
 
 
 
 
 
 type 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin

In [10]:
val adoptionRate = years.map { year ->
    (cumulativeKotlinRepos.filter { it["year"] == year }[0]["yearly_count"].toString().toFloat() /
            cumulativeJvmRepos.filter { it["year"] == year }[0]["yearly_count"].toString().toFloat() ) * 100
}

val adoptionRateWithStart = adoptionRate.toMutableList()
adoptionRateWithStart.add(0, 0F)
val yearsWithStart = years.toMutableList()
yearsWithStart.add(0, "2018")

val adoptionRateFrame = dataFrameOf(
    "year" to yearsWithStart,
    "rate" to adoptionRateWithStart
)

adoptionRateFrame.plot() {
    line {
        x("year"){
            axis.name = "Calendar year"
        }
        y("rate"){
            axis.name = "Percent"
        }
        color = Color.ORANGE
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="TRxA5s"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"year":["2018","2019","2020","2021","2022","2023","2024"],
"rate":[0.0,6.560364246368408,6.4453125,8.32025146484375,6.723891258239746,10.746606826782227,11.052631378173828]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Calendar year"
},{
"aesthetic":"y",
"name":"Percent",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"year",
"y":"rate"
},
"stat":"identity",
"color":"#fc8452",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
}]
};
 var plotContainer = document.getElementById("TRxA5s");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2018 
 
 
 
 
 
 
 
 
 2019 
 
 
 
 
 
 
 
 
 2020 
 
 
 
 
 
 
 
 
 2021 
 
 
 
 
 
 
 
 
 2022 
 
 
 
 
 
 
 
 
 2023 
 
 
 
 
 
 
 
 
 2024 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 Percent 
 
 
 
 
 Calendar year

In [33]:
// We want the distribution of different types of Kotlin repos

val kotlinRepos = jvm_active_repositories.filter { it["is_kotlin"] == 1}

val androidRepos = kotlinRepos.filter { it["is_android"] == 1 }
val javaNonAndroid = kotlinRepos.filter { it["is_android"] == 0 && it["is_java"] == 1 }
val pureKotlinRepos = kotlinRepos.filter { it["is_android"] == 0 && it["is_java"] == 0 }

// Validating I captured everything
println(androidRepos.rowsCount() + javaNonAndroid.rowsCount() + pureKotlinRepos.rowsCount() )


val repository_type by columnOf("Android", "Pure Kotlin", "Other")
val count by columnOf(androidRepos.rowsCount(), pureKotlinRepos.rowsCount(), javaNonAndroid.rowsCount())
val df = dataFrameOf(repository_type, count)

df.plot {
    pie {
        slice(count)
        fillColor(repository_type) {
            scale = categorical(
                "Android" to Color.GREEN,
                "Pure Kotlin" to Color.PURPLE,
                "Other" to Color.ORANGE
            )
        }
        size = 33.0
        hole = 0.8
        alpha = 0.8
    }
    layout {
        style(Style.Void)
    }
}




435


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="iDibOa"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"count":[137.0,177.0,121.0],
"repository_type":["Android","Pure Kotlin","Other"]
},
"kind":"plot",
"scales":[{
"aesthetic":"fill",
"values":["#3ba272","#9a60b4","#fc8452"],
"limits":["Android","Pure Kotlin","Other"]
}],
"layers":[{
"hole":0.8,
"mapping":{
"slice":"count",
"fill":"repository_type"
},
"stat":"identity",
"size":33.0,
"sampling":"none",
"alpha":0.8,
"position":"identity",
"geom":"pie",
"data":{
}
}],
"theme":{
"name":"classic",
"axis":{
"blank":true
},
"line":{
"blank":true
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
}
};
 var plotContainer = document.getElementById("iDibOa");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 repository_type 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Android 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Pure Kotlin 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Other

In [34]:
import java.time.LocalDateTime
import java.time.LocalDate

// Here we want to see the ratio JVM / Kotlin segregated by location

val nonEuropeKotlinRepos = kotlinRepos.filter{ it["location"] == "World"}
val europeKotlinRepos = kotlinRepos.filter{ it["location"] == "Europe"}

println(kotlinRepos.rowsCount())
println(nonEuropeKotlinRepos.rowsCount())
println(europeKotlinRepos.rowsCount())

val europeRatio = (europeKotlinRepos.rowsCount().toDouble() / kotlinRepos.rowsCount().toDouble()) * 100

println("Overall the ratio is : ${europeRatio.toInt()}% in Europe and ${(100 - europeRatio).toInt()}% outside")

// Ok and now only for last year
val now = LocalDateTime.now()
val aYearAgo = now.minusYears(1)

println(jvm_active_repositories.rowsCount())

val jvmRepos2024 = jvm_active_repositories.filter {
    it["oldest_commit"] != null &&
            LocalDate.parse(it["oldest_commit"].toString().split(" ")[0]).atTime(23, 59) > aYearAgo
}

println(jvmRepos2024.rowsCount())

val jvmReposEurope2024 = jvmRepos2024.filter{ it["location"] == "INGEurCDaaS01"}
val jvmReposNonEurope2024 = jvmRepos2024.filter{ it["location"] == "IngCDaaS"}

println(jvmReposEurope2024.rowsCount())
println(jvmReposNonEurope2024.rowsCount())

println("###")
val nonEuropeKotlinRepos2024 = jvmReposNonEurope2024.filter{ it["is_kotlin"] == 1}
val europeKotlinRepos2024 = jvmReposEurope2024.filter{ it["is_kotlin"] == 1}

println(europeKotlinRepos2024.rowsCount())
println(nonEuropeKotlinRepos2024.rowsCount())

val europeKotlinRatio2024 = (europeKotlinRepos2024.rowsCount().toDouble() / jvmReposEurope2024.rowsCount().toDouble()) * 100
val nonEuropeKotlinRatio2024 = (nonEuropeKotlinRepos2024.rowsCount().toDouble() / jvmReposNonEurope2024.rowsCount().toDouble()) * 100

println("Finally the ratio is : ${europeKotlinRatio2024.toInt()}% in Europe and ${nonEuropeKotlinRatio2024.toInt()}% outside")


435
0
0
Overall the ratio is : 0% in Europe and 100% outside
5497
533
414
119
###
46
14
Finally the ratio is : 11% in Europe and 11% outside
